# Run Ilastik segmentation and TrackMate tracking

This notebook runs Ilastik segmentation and TrackMate tracking

To work correctly, it requires the following input:
- A metadata ***.csv*** containing information on every sample to be processed
- A 5D (ctzyx) ***.tiff*** file of the raw imaging data




In [1]:
from behav3d.preprocessing.segmentation.ilastik_segmentation import run_ilastik_segmentation
from behav3d.preprocessing.tracking.trackmate_tracking import (
    run_trackmate_tcells, 
    run_trackmate_organoids,
    run_trackmate
)
import pandas as pd
from pathlib import Path

### Load metadata and supply Ilastik paths


In [2]:
### Path for the output files generated by the pipeline
output_dir = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/behav3d_output"
# PATH TO THE METADATA CSV
metadata_csv_path = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/metadata.csv"

# PATH TO ILASTIK
ilastik_path = "/Applications/ilastik-1.4.0b27-OSX.app/Contents/ilastik-release/run_ilastik.sh"

# MODEL PATHS
ilastik_pixel_classifier_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_pixel_class.ilp"
ilastik_organoid_segmentation_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_organoid_class.ilp"
ilastik_tcell_segmentation_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_tcell_class.ilp"
ilastik_organoid_postprocessing_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_object_splitter.ilp"
ilastik_tcell_postprocessing_model = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_object_splitter.ilp"

### Read in the metadata file and display it
metadata = pd.read_csv(metadata_csv_path)
print("Metadata provided:")
metadata

Metadata provided:


,sample_name,organoid_line,tcell_line,exp_nr,well,dead_dye_channel,dead_dye_threshold,contact_threshold,pixel_distance_xy,pixel_distance_z,distance_unit,time_interval,time_unit,raw_image_path,tcell_segments_path,organoid_segments_path,tcell_track_csv
0,AIM_MB2_Exp58_Img003_donor899,10T,CART_WT1,1,well0,3,3000,3.6,1.38,1.38,μm,2,m,/Users/samdeblank/Documents/1.projects/BHVD_BE...,/Users/samdeblank/Documents/1.projects/BHVD_BE...,/Users/samdeblank/Documents/1.projects/BHVD_BE...,/Users/samdeblank/Documents/1.projects/BHVD_BE...


### Check correct usage of metadata.csv

Checks the supplied metadata .csv for:
- Are all columns filled in?
- Does teh raw_image path exist?

In [3]:
assert not any(metadata.isna().any()), "Some column values have not been supplied. Make sure you correctly supply values for all columns in the metadata .csv"

for rowidx, sample_metadata in metadata.iterrows():
    sample_name = sample_metadata['sample_name']
    assert Path(sample_metadata["raw_image_path"]).exists(), f"The image_path supplied for Row {rowidx+1} '{sample_name}' does not exist"
print("Metadata file is OK!")

Metadata file is OK!


---

# Performing ilastik segmentation

---

Uses the pretrained supplied ilastik models for pixel classification, object detection and object postprocessing (object splitting) for both T cells and Organoids/Tumor cells

*Duration: 2 hours*<br>
*On filesize: 30Gb*

Pixel classifier: ~10 hours

Organoid segmentation: ~ 35 minutes

T cell segmentation: ~ 35 minutes


In [5]:
run_ilastik_segmentation(
    output_dir=output_dir,
    metadata=metadata,
    ilastik_path = ilastik_path,
    ilastik_pix_clas_model = ilastik_pixel_classifier_model,
    ilastik_org_seg_model = ilastik_organoid_segmentation_model,
    ilastik_tcell_seg_model = ilastik_tcell_segmentation_model,
    ilastik_org_postproc_model = ilastik_organoid_postprocessing_model,
    ilastik_tcell_postproc_model = ilastik_tcell_postprocessing_model, 
    verbose=False,
    keep_all=True
    )

--------------- Using existing raw_img .h5: AIM_MB2_Exp58_Img003_donor899 ---------------
/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/behav3d_output/images/AIM_MB2_Exp58_Img003_donor899/AIM_MB2_Exp58_Img003_donor899_ilastik_input.h5
--------------- Segmenting: AIM_MB2_Exp58_Img003_donor899 ---------------
- Running the ilastik pixel classifier...


KeyboardInterrupt: 

---

# Performing TrackMate tracking

---

Applies TrackMate for the tracking of the T cells and organoids/tumor cells over all timepoints

*NOTE* If giving an error, restarting the notebook might fix the issue (especially if you have run and canceled this block in the current session)

*Duration: ~5 minutes*


In [4]:
run_trackmate(
    output_dir=output_dir, 
    metadata=metadata, 
    verbose=False, 
    cell_type="tcells"
    )

--------------- Tracking (tcells) : AIM_MB2_Exp58_Img003_donor899 ---------------
- Running TrackMate...
Setting 14 Gb of memory based on available memory


Operating in headless mode - the original ImageJ will have limited functionality.
Operating in headless mode - the IJ class will not be fully functional.
Operating in headless mode - the WindowManager class will not be fully functional.


Starting detection process using 16 threads.

Detection processes 16 frames simultaneously and allocates 1 thread per frame.



[java.lang.Enum.toString] Starting detection process using 16 threads.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Detection processes 16 frames simultaneously and allocates 1 thread per frame.
[java.lang.Enum.toString] 


Found 30667 spots.

Starting initial filtering process.

Computing spot features over 16 frames simultaneously and allocating 1 thread per frame.

Computation done in 11 ms.

Starting spot filtering process.

Starting tracking process.



[java.lang.Enum.toString] Found 30667 spots.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Starting initial filtering process.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computing spot features over 16 frames simultaneously and allocating 1 thread per frame.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computation done in 11 ms.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Starting spot filtering process.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Starting tracking process.
[java.lang.Enum.toString] 


Computing edge features:

Computation done in 1 ms.

Computing track features:

Computation done in 8 ms.

Starting track filtering process.



[java.lang.Enum.toString] Computing edge features:
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computation done in 1 ms.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computing track features:
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computation done in 8 ms.
[java.lang.Enum.toString] 
[java.lang.Enum.toString] Starting track filtering process.
[java.lang.Enum.toString] 
/Users/samdeblank/surfdrive/Documents/github/BEHAV3D-opensource/behav3d/preprocessing/tracking/trackmate_tracking.py:205: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_spots = pd.concat([df_spots, new_row], ignore_index=True)


- Assigning track ID to segmented image to create tracked image...
0 12.433398058252424 258.19398058252426 29.70796116504854
1 12.49031847133758 257.4930573248407 30.57095541401274
2 12.36888888888889 259.4771717171717 33.041010101010095
3 12.429169435215949 259.60504983388705 32.37269102990033
4 12.329896907216494 258.9847422680412 31.71628865979381
5 12.463396226415094 259.33584905660376 33.35433962264151
6 12.64272151898734 259.82867088607594 33.002088607594935
7 12.384155844155844 260.1972077922078 33.258896103896106
8 12.44653846153846 260.21403846153845 33.09788461538461
9 12.498857142857142 259.8255238095238 33.14628571428571
10 12.72996923076923 259.4017846153846 35.96492307692307
11 12.507818181818182 259.624 34.658909090909084
12 12.363584905660376 261.40150943396225 34.170188679245285
13 12.514874999999998 261.6825 35.3581875
14 12.382590361445782 262.50343373493973 34.882409638554215
15 12.293394495412842 262.5629357798165 34.50844036697247
16 12.540974025974023 264.0594155

In [5]:
run_trackmate(
    output_dir=output_dir, 
    metadata=metadata, 
    verbose=False, 
    cell_type="organoids"
    )

--------------- Tracking (organoids) : AIM_MB2_Exp58_Img003_donor899 ---------------
- Running TrackMate...
Setting 8 Gb of memory based on available memory


Operating in headless mode - the original ImageJ will have limited functionality.
Operating in headless mode - the IJ class will not be fully functional.
Operating in headless mode - the WindowManager class will not be fully functional.


TypeError: Ambiguous overloads found for ij.IJ.run(NoneType,str,str) between:
	public static void ij.IJ.run(ij.ImagePlus,java.lang.String,java.lang.String)
	public static void ij.IJ.run(ij.macro.Interpreter,java.lang.String,java.lang.String)

